In [136]:
with open('log_linkx_snap.txt', 'r') as f:
    all_logs = f.readlines() 

In [137]:
import re

method_list = []
dataset_list = []
lr_list = []
epochs_list = []
hidden_list = []
dropout_list = []
weight_decay_list = []
alpha_list = []
beta_list = []
gamma_list = []
norm_func_id_list = []
norm_layers_list = []
orders_func_id_list = []
orders_list = []
highest_train_list = []
highest_valid_list = []
final_train_list = []
final_test_list = []
delta_list = []
final_test_std_list = []
num_layers_list = []


dataset_tag = False
final_test_tag = False


for logs in all_logs:
    if 'Namespace' in logs:
        if dataset_tag == True and final_test_tag == False:
            highest_train_list.append('0')
            highest_valid_list.append('0')
            final_train_list.append('0')
            final_test_list.append('0')
            final_test_std_list.append('0')

        method_list.extend(re.findall(r"method='(.*?)', no_bn", logs))
        dataset_list.extend(re.findall(r"dataset='(.*?)', decay_rate", logs))
        delta_list.extend(re.findall(r"delta=(.*?), directed", logs))
        lr_list.extend(re.findall(r"lr=(.*?), method", logs))
        epochs_list.extend(re.findall(r"epochs=(.*?), exponent", logs))
        dropout_list.extend(re.findall(r"dropout=(.*?), epochs", logs))
        weight_decay_list.extend(re.findall(r"weight_decay=(.*?)\)", logs))
        alpha_list.extend(re.findall(r"alpha=(.*?), beta", logs))
        beta_list.extend(re.findall(r"beta=(.*?), cached", logs))
        orders_func_id_list.extend(re.findall(r"orders_func_id=(.*?), print_prop", logs))
        orders_list.extend(re.findall(r"orders=(.*?), orders_func_id", logs))
        gamma_list.extend(re.findall(r"gamma=(.*?), gat_heads", logs))
        norm_func_id_list.extend(re.findall(r"norm_func_id=(.*?), norm_layers", logs))
        norm_layers_list.extend(re.findall(r"norm_layers=(.*?), num_layers", logs))
        hidden_list.extend(re.findall(r"hidden_channels=(.*?), hops", logs))
        num_layers_list.extend(re.findall(r"num_layers=(.*?), num_mlp_layers", logs))


        dataset_tag = True
        final_test_tag = False

        
    if 'Highest Train' in logs and ',' in logs:
        highest_train_list.extend(re.findall(r"Highest Train: (.*?),", logs))
    
    if 'Highest Valid' in logs and ',' in logs:
        highest_valid_list.extend(re.findall(r"Highest Valid: (.*?),", logs))
        
    if 'Final Train' in logs and ',' in logs:
        final_train_list.extend(re.findall(r"Final Train: (.*?),", logs))
        
    if 'Final Test' in logs and ',' in logs:
        final_test_list.extend(re.findall(r"Final Test: (.*?),", logs))
        final_test_std_list.extend(re.findall(r", (.*?)\n", logs))

        final_test_tag = True
        dataset_tag = False


if dataset_tag == True and final_test_tag == False:
        highest_train_list.append('0')
        highest_valid_list.append('0')
        final_train_list.append('0')
        final_test_list.append('0')
        final_test_std_list.append('0')
    
        

In [138]:
import pandas as pd
import os

d = {
    'method': method_list,
    'dataset': dataset_list,
    'lr': lr_list,
    'epoch': epochs_list,
    'hidden': hidden_list,
    'num_layers': num_layers_list,
    'dropout': dropout_list,
    'weight_decay': weight_decay_list,
    'alpha': alpha_list,
    'beta': beta_list,
    'gamma': gamma_list,
    'delta': delta_list,
    'norm_layers': norm_layers_list,
    'orders': orders_list,
    'high_train': highest_train_list,
    'high_valid': highest_valid_list,
    'final_train': final_train_list,
    'final_test': final_test_list,
    'final_std_test': final_test_std_list
}

for k, v in d.items():
    print(k, len(v))

csv_name = 'results/large_dataset_tuning14.csv'
df = pd.DataFrame.from_dict(d)
df = df.sort_values(['method', 'final_test']).reset_index(drop=True)
# if os.path.exists(csv_name):
#     df.to_csv(csv_name, mode='a', header=False)
# else:
#     df.to_csv(csv_name)
df[-10:]

method 12
dataset 12
lr 12
epoch 12
hidden 12
num_layers 12
dropout 12
weight_decay 12
alpha 12
beta 12
gamma 12
delta 12
norm_layers 12
orders 12
high_train 12
high_valid 12
final_train 12
final_test 12
final_std_test 12


,method,dataset,lr,epoch,hidden,num_layers,dropout,weight_decay,alpha,beta,gamma,delta,norm_layers,orders,high_train,high_valid,final_train,final_test,final_std_test
2,linkx,snap-patents,0.01,500,256,3,0.5,0.001,0.0,1.0,0.0,0.0,1,1,0,0,0,0,0
3,linkx,snap-patents,0.01,500,16,2,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.11,51.61,84.34,51.57,2.76
4,linkx,snap-patents,0.01,500,16,3,0.5,0.001,0.0,1.0,0.0,0.0,1,1,88.95,53.88,86.73,53.84,1.04
5,linkx,snap-patents,0.01,500,32,3,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.21,55.30,86.27,55.27,1.28
6,linkx,snap-patents,0.01,500,32,2,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.23,55.37,86.83,55.35,2.09
7,linkx,snap-patents,0.01,500,128,2,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.29,57.60,89.07,57.57,0.44
8,linkx,snap-patents,0.01,500,128,3,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.31,57.81,87.61,57.79,0.43
9,linkx,snap-patents,0.01,500,16,1,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.08,61.16,88.68,61.16,0.25
10,linkx,snap-patents,0.01,500,32,1,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.10,61.73,88.63,61.72,0.23
11,linkx,snap-patents,0.01,500,128,1,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.26,62.10,87.93,62.10,0.11


In [139]:
df

,method,dataset,lr,epoch,hidden,num_layers,dropout,weight_decay,alpha,beta,gamma,delta,norm_layers,orders,high_train,high_valid,final_train,final_test,final_std_test
0,linkx,snap-patents,0.01,500,256,1,0.5,0.001,0.0,1.0,0.0,0.0,1,1,0,0,0,0,0
1,linkx,snap-patents,0.01,500,256,2,0.5,0.001,0.0,1.0,0.0,0.0,1,1,0,0,0,0,0
2,linkx,snap-patents,0.01,500,256,3,0.5,0.001,0.0,1.0,0.0,0.0,1,1,0,0,0,0,0
3,linkx,snap-patents,0.01,500,16,2,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.11,51.61,84.34,51.57,2.76
4,linkx,snap-patents,0.01,500,16,3,0.5,0.001,0.0,1.0,0.0,0.0,1,1,88.95,53.88,86.73,53.84,1.04
5,linkx,snap-patents,0.01,500,32,3,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.21,55.30,86.27,55.27,1.28
6,linkx,snap-patents,0.01,500,32,2,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.23,55.37,86.83,55.35,2.09
7,linkx,snap-patents,0.01,500,128,2,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.29,57.60,89.07,57.57,0.44
8,linkx,snap-patents,0.01,500,128,3,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.31,57.81,87.61,57.79,0.43
9,linkx,snap-patents,0.01,500,16,1,0.5,0.001,0.0,1.0,0.0,0.0,1,1,89.08,61.16,88.68,61.16,0.25
